In [1]:
import numpy as np
import os 
import pandas as pd
os.environ["TF_ENABLE_ONEDNN_OPTS"]="0"
import tensorflow as tf
import tensorflow_datasets as tfds
keras = tf.keras

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
validation_ds = tf.keras.utils.image_dataset_from_directory(
    scaled_wo_rotate_validation, 
    labels = 'inferred', 
    image_size=(img_height,img_width)
    )

Found 43733 files belonging to 6 classes.


2022-08-01 12:17:57.272955: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 12:17:57.281499: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 12:17:57.284216: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 12:17:57.287696: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [5]:
training_ds = tf.keras.utils.image_dataset_from_directory(
    scaled_wo_rotate_training, 
    labels = 'inferred', 
    image_size=(img_height,img_width)
    )

Found 349639 files belonging to 6 classes.


In [6]:
n_class = 6
base_model = tf.keras.applications.MobileNetV3Small(input_shape = (224,224,3), include_top = False, weights='imagenet')
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_class, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [7]:
for layer in base_model.layers:
    layer.trainable=False

In [8]:
model.compile(optimizer="Adam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

 expanded_conv_1/depthwise/Batc  (None, 28, 28, 72)  288         ['expanded_conv_1/depthwise[0][0]
 hNorm (BatchNormalization)                                      ']                               
                                                                                                  
 re_lu_4 (ReLU)                 (None, 28, 28, 72)   0           ['expanded_conv_1/depthwise/Batch
                                                                 Norm[0][0]']                     
                                                                                                  
 expanded_conv_1/project (Conv2  (None, 28, 28, 24)  1728        ['re_lu_4[0][0]']                
 D)                                                                                               
                                                                                                  
 expanded_conv_1/project/BatchN  (None, 28, 28, 24)  96          ['expanded_conv_1/project[0][0]']
 orm (Batc

                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 24)    2328        ['expanded_conv_3/squeeze_excite/
 /Conv (Conv2D)                                                  AvgPool[0][0]']                  
                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 24)    0           ['expanded_conv_3/squeeze_excite/
 /Relu (ReLU)                                                    Conv[0][0]']                     
                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 96)    2400        ['expanded_conv_3/squeeze_excite/
 /Conv_1 (Conv2D)                                                Relu[0][0]']                     
                                                                                                  
 tf.__oper

                                                                                                  
 expanded_conv_4/squeeze_excite  (None, 14, 14, 240)  0          ['multiply_4[0][0]',             
 /Mul (Multiply)                                                  'tf.math.multiply_7[0][0]']     
                                                                                                  
 expanded_conv_4/project (Conv2  (None, 14, 14, 40)  9600        ['expanded_conv_4/squeeze_excite/
 D)                                                              Mul[0][0]']                      
                                                                                                  
 expanded_conv_4/project/BatchN  (None, 14, 14, 40)  160         ['expanded_conv_4/project[0][0]']
 orm (BatchNormalization)                                                                         
                                                                                                  
 expanded_

                                                                 rm[0][0]']                       
                                                                                                  
 expanded_conv_6/expand (Conv2D  (None, 14, 14, 120)  4800       ['expanded_conv_5/Add[0][0]']    
 )                                                                                                
                                                                                                  
 expanded_conv_6/expand/BatchNo  (None, 14, 14, 120)  480        ['expanded_conv_6/expand[0][0]'] 
 rm (BatchNormalization)                                                                          
                                                                                                  
 tf.__operators__.add_11 (TFOpL  (None, 14, 14, 120)  0          ['expanded_conv_6/expand/BatchNor
 ambda)                                                          m[0][0]']                        
          

                                                                 m[0][0]',                        
                                                                  'tf.math.multiply_14[0][0]']    
                                                                                                  
 expanded_conv_7/depthwise (Dep  (None, 14, 14, 144)  3600       ['multiply_9[0][0]']             
 thwiseConv2D)                                                                                    
                                                                                                  
 expanded_conv_7/depthwise/Batc  (None, 14, 14, 144)  576        ['expanded_conv_7/depthwise[0][0]
 hNorm (BatchNormalization)                                      ']                               
                                                                                                  
 tf.__operators__.add_15 (TFOpL  (None, 14, 14, 144)  0          ['expanded_conv_7/depthwise/Batch
 ambda)   

 tf.__operators__.add_18 (TFOpL  (None, 7, 7, 288)   0           ['expanded_conv_8/depthwise/Batch
 ambda)                                                          Norm[0][0]']                     
                                                                                                  
 re_lu_23 (ReLU)                (None, 7, 7, 288)    0           ['tf.__operators__.add_18[0][0]']
                                                                                                  
 tf.math.multiply_18 (TFOpLambd  (None, 7, 7, 288)   0           ['re_lu_23[0][0]']               
 a)                                                                                               
                                                                                                  
 multiply_12 (Multiply)         (None, 7, 7, 288)    0           ['expanded_conv_8/depthwise/Batch
                                                                 Norm[0][0]',                     
          

 /Conv (Conv2D)                                                  AvgPool[0][0]']                  
                                                                                                  
 expanded_conv_9/squeeze_excite  (None, 1, 1, 144)   0           ['expanded_conv_9/squeeze_excite/
 /Relu (ReLU)                                                    Conv[0][0]']                     
                                                                                                  
 expanded_conv_9/squeeze_excite  (None, 1, 1, 576)   83520       ['expanded_conv_9/squeeze_excite/
 /Conv_1 (Conv2D)                                                Relu[0][0]']                     
                                                                                                  
 tf.__operators__.add_22 (TFOpL  (None, 1, 1, 576)   0           ['expanded_conv_9/squeeze_excite/
 ambda)                                                          Conv_1[0][0]']                   
          

                                                                                                  
 tf.math.multiply_25 (TFOpLambd  (None, 1, 1, 576)   0           ['re_lu_30[0][0]']               
 a)                                                                                               
                                                                                                  
 expanded_conv_10/squeeze_excit  (None, 7, 7, 576)   0           ['multiply_16[0][0]',            
 e/Mul (Multiply)                                                 'tf.math.multiply_25[0][0]']    
                                                                                                  
 expanded_conv_10/project (Conv  (None, 7, 7, 96)    55296       ['expanded_conv_10/squeeze_excite
 2D)                                                             /Mul[0][0]']                     
                                                                                                  
 expanded_

In [10]:
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

In [11]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

In [12]:
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def resized(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    return resized_image, label

In [13]:
from functools import partial

In [14]:
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(resized, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(resized).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(resized).batch(batch_size).prefetch(1)

In [15]:
dataset_size = info.splits["train"].num_examples

In [17]:
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=10)

Epoch 1/10


2022-08-01 12:19:07.308375: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


86/86 [==============================] - 9s 44ms/step - loss: 0.9632 - accuracy: 0.6562 - val_loss: 0.6248 - val_accuracy: 0.7923
Epoch 2/10
86/86 [==============================] - 3s 29ms/step - loss: 0.5031 - accuracy: 0.8463 - val_loss: 0.4961 - val_accuracy: 0.8474
Epoch 3/10
86/86 [==============================] - 2s 29ms/step - loss: 0.3977 - accuracy: 0.8775 - val_loss: 0.4565 - val_accuracy: 0.8585
Epoch 4/10
86/86 [==============================] - 2s 29ms/step - loss: 0.3457 - accuracy: 0.8961 - val_loss: 0.4134 - val_accuracy: 0.8768
Epoch 5/10
86/86 [==============================] - 2s 29ms/step - loss: 0.3147 - accuracy: 0.8935 - val_loss: 0.4000 - val_accuracy: 0.8695
Epoch 6/10
86/86 [==============================] - 2s 29ms/step - loss: 0.2876 - accuracy: 0.9062 - val_loss: 0.3865 - val_accuracy: 0.8805
Epoch 7/10
86/86 [==============================] - 2s 29ms/step - loss: 0.2749 - accuracy: 0.9081 - val_loss: 0.3781 - val_accuracy: 0.8768
Epoch 8/10
86/86 [======

In [18]:
for layer in base_model.layers:
    layer.trainable=True

In [19]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

In [20]:
model.compile(optimizer=opt,
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [21]:
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=200)

Epoch 1/200
86/86 [==============================] - 11s 68ms/step - loss: 0.4363 - accuracy: 0.8477 - val_loss: 0.3467 - val_accuracy: 0.8915
Epoch 2/200
86/86 [==============================] - 5s 60ms/step - loss: 0.3610 - accuracy: 0.8739 - val_loss: 0.3432 - val_accuracy: 0.8952
Epoch 3/200
86/86 [==============================] - 5s 60ms/step - loss: 0.3458 - accuracy: 0.8826 - val_loss: 0.3392 - val_accuracy: 0.8897
Epoch 4/200
86/86 [==============================] - 5s 60ms/step - loss: 0.3193 - accuracy: 0.8943 - val_loss: 0.3353 - val_accuracy: 0.8897
Epoch 5/200
86/86 [==============================] - 5s 60ms/step - loss: 0.3042 - accuracy: 0.8986 - val_loss: 0.3298 - val_accuracy: 0.8934
Epoch 6/200
86/86 [==============================] - 5s 60ms/step - loss: 0.2885 - accuracy: 0.9033 - val_loss: 0.3261 - val_accuracy: 0.8897
Epoch 7/200
86/86 [==============================] - 5s 60ms/step - loss: 0.2708 - accuracy: 0.9073 - val_loss: 0.3220 - val_accuracy: 0.8934
Epoch

86/86 [==============================] - 5s 62ms/step - loss: 0.0580 - accuracy: 0.9873 - val_loss: 0.2578 - val_accuracy: 0.9246
Epoch 59/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0519 - accuracy: 0.9862 - val_loss: 0.2586 - val_accuracy: 0.9246
Epoch 60/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0550 - accuracy: 0.9869 - val_loss: 0.2591 - val_accuracy: 0.9210
Epoch 61/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0530 - accuracy: 0.9851 - val_loss: 0.2590 - val_accuracy: 0.9210
Epoch 62/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0512 - accuracy: 0.9858 - val_loss: 0.2594 - val_accuracy: 0.9228
Epoch 63/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0547 - accuracy: 0.9862 - val_loss: 0.2595 - val_accuracy: 0.9228
Epoch 64/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0528 - accuracy: 0.9873 - val_loss: 0.2599 - val_accuracy: 0.9228
Epoch 65/200

86/86 [==============================] - 5s 62ms/step - loss: 0.0248 - accuracy: 0.9938 - val_loss: 0.2811 - val_accuracy: 0.9210
Epoch 116/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0183 - accuracy: 0.9967 - val_loss: 0.2808 - val_accuracy: 0.9228
Epoch 117/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0211 - accuracy: 0.9953 - val_loss: 0.2831 - val_accuracy: 0.9191
Epoch 118/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0147 - accuracy: 0.9982 - val_loss: 0.2857 - val_accuracy: 0.9191
Epoch 119/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0180 - accuracy: 0.9956 - val_loss: 0.2896 - val_accuracy: 0.9210
Epoch 120/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0205 - accuracy: 0.9935 - val_loss: 0.2886 - val_accuracy: 0.9228
Epoch 121/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0146 - accuracy: 0.9964 - val_loss: 0.2891 - val_accuracy: 0.9228
Epoch 

Epoch 172/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0091 - accuracy: 0.9982 - val_loss: 0.3089 - val_accuracy: 0.9283
Epoch 173/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0072 - accuracy: 0.9993 - val_loss: 0.3106 - val_accuracy: 0.9301
Epoch 174/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0095 - accuracy: 0.9971 - val_loss: 0.3056 - val_accuracy: 0.9283
Epoch 175/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0097 - accuracy: 0.9978 - val_loss: 0.3040 - val_accuracy: 0.9301
Epoch 176/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0076 - accuracy: 0.9982 - val_loss: 0.3080 - val_accuracy: 0.9283
Epoch 177/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0092 - accuracy: 0.9982 - val_loss: 0.3083 - val_accuracy: 0.9301
Epoch 178/200
86/86 [==============================] - 5s 62ms/step - loss: 0.0086 - accuracy: 0.9985 - val_loss: 0.3087 - val_accuracy:

In [22]:
model.evaluate(test_set)

12/12 [==============================] - 1s 41ms/step - loss: 0.2125 - accuracy: 0.9373


[0.21250896155834198, 0.9373297095298767]

In [23]:
hist_df = pd.DataFrame(history.history)
hist = 'history.csv' 
with open(hist, mode='w') as f:
    hist_df.to_csv(f)